In [1]:
import proposition as p
import p_symbol as ps
import sympy as sp

## Proving $x^2$ is continuous at $x=0$

In [2]:
x = ps.Symbol("x", real=True)
eps = ps.Symbol(r"\epsilon", real=True)

In [3]:
eps_positive = p.GreaterThan(eps, 0, is_assumption=True)
eps_positive

\epsilon > 0

In [4]:
absolute_x_positive = p.GreaterThan.is_absolute(sp.Abs(x))
absolute_x_positive

Abs(x) > 0

In [5]:
root_eps_positive = p.GreaterThan.is_rational_power(sp.sqrt(eps), eps_positive)
root_eps_positive

sqrt(\epsilon) > 0

In [6]:
absx_lt_sqrt_eps = p.LessThan(sp.Abs(x), sp.sqrt(eps), is_assumption=True)
absx_lt_sqrt_eps

Abs(x) < sqrt(\epsilon)

In [7]:
xsq_lt_eps_t_absx = absx_lt_sqrt_eps.p_multiply_by_positive(
        abs(x), absolute_x_positive
    )
xsq_lt_eps_t_absx

x**2 < sqrt(\epsilon)*Abs(x)

In [8]:
eps_t_absx_lt_eps = absx_lt_sqrt_eps.p_multiply_by_positive(
        sp.sqrt(eps), root_eps_positive
    )
xsq_lt_eps = xsq_lt_eps_t_absx.transitive(eps_t_absx_lt_eps)
xsq_lt_eps

x**2 < \epsilon

In [9]:
xsq_lt_eps.is_proven

True

In a written proof, we would probably stop here, as we've proven that given some arbitrary $\epsilon$, we have a value which is in fact $\sqrt \epsilon$ that makes $x^2 > \epsilon$. However, we can construct the actual statement we wanted to prove from these parts.

The statement is somewhat backwards, since we are constructing it from inside out. I chose this style because it is
somewhat readable without too many nested parentheses.

In [10]:
lim_x_sq_as_x_tends_to_0_eq_0 = (
    xsq_lt_eps.followed_from(absx_lt_sqrt_eps)
    .p_and_reverse(root_eps_positive)
    .thus_there_exists("delta", sp.sqrt(eps), [[0], [1, 0]])
    .followed_from(eps_positive)
    .thus_forall(eps)
)

In [11]:
lim_x_sq_as_x_tends_to_0_eq_0

forall \epsilon: [\epsilon > 0 -> exists delta: (delta > 0 /\ [Abs(x) < delta -> x**2 < \epsilon])]

In [12]:
lim_x_sq_as_x_tends_to_0_eq_0.is_proven

True

## Proving $(A+B)^T = A^T + B^T$

In [13]:
m, n, i, j = ps.symbols("m n i j", integer=True)
A = ps.MatrixSymbol("A", m, n)
B = ps.MatrixSymbol("B", m, n)
R = ps.MatrixSymbol("R", m, n)
R_eq_A_plus_B = p.Equals(R, A + B, is_assumption=True)

In [14]:
R_eq_A_plus_B

R = A + B

In [15]:
to_prove = p.Equals(R.T, A.T + B.T)
to_prove

R.T = A.T + B.T

The line with `by_simplification` is essentially what we wanted to prove, but I'm showcasing the `p_substitute_into` tactic which substitutes a proven equation, in this case the left side of `R = A + B` into another proposition.

In [16]:
a_plus_b_transpose_eq_a_t_plus_b_t = p.Equals((A+B).T, A.T + B.T).by_simplification()
res = R_eq_A_plus_B.p_substitute_into("left", a_plus_b_transpose_eq_a_t_plus_b_t)
res

R.T = A.T + B.T

In [17]:
res.is_proven

True

## Proving $(AB)^T = B^T A^T$

In [18]:
m, n, r, i, i1, j, k = ps.symbols("m n r i i_1 j k", integer=True)
A = ps.MatrixSymbol("A", m, n)
B = ps.MatrixSymbol("B", n, r)
to_prove = p.Equals((A*B).T, B.T * A.T)

In [19]:
to_prove

(A*B).T = B.T*A.T

`by_simplification` also works directly here.

In [20]:
to_prove.by_simplification().is_proven

True

But we can show that each corresponding component is equal in both matrices, and then use `hence_matrices_are_equal` which basically says, "If all corresponding components of two matrices are equal, the matrices are equal".

In [21]:
components_eq = p.Equals((A*B).T[i, j], (B.T * A.T)[i,j])
ab_T_equals_b_T_a_T = (
    components_eq.by_simplification()
    .thus_forall(j)
    .thus_forall(i)
    .hence_matrices_are_equal()
    )
ab_T_equals_b_T_a_T

(A*B).T = B.T*A.T

In [22]:
ab_T_equals_b_T_a_T.is_proven

True